# Biostat M280 Homework 4

**Due June 12 @ 11:59PM**

In this homework, we build a classifer for handwritten digit recognition. Following figure shows example bitmaps of handwritten digits from U.S. postal envelopes. 

Each digit is represented by a $32 \times 32$ bitmap in which each element indicates one pixel with a value of white or black. Each $32 \times 32$ bitmap is divided into blocks of $4 \times 4$, and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector $\mathbf{x} = (x_1, \ldots, x_{64})$ of length 64 where each element is a count between 0 and 16. 

We will use a model-based method by assuming a distribution on the count vector and carry out classification using probabilities. A common distribution for count vectors is the multinomial distribution. However as you will see in Q10, it is not a good model for handwritten digits. Let's work on a more flexible model for count vectors. In the Dirichlet-multinomial model, we assume the multinomial probabilities $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$, and density
$$
\begin{eqnarray*}
	\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1},
\end{eqnarray*} 
$$
where $|\alpha|=\sum_{j=1}^d \alpha_j$.

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


$$ Pr(\mathbf{x}\mid\boldsymbol{\alpha})=\int_{\mathbf{p}}\Pr(\mathbf{x}\mid \mathbf{p})\Pr(\mathbf{p}\mid\boldsymbol{\alpha})\textrm{d}\mathbf{p} $$,
$$ = \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p} $$,
where $\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}$
$$Pr(\mathbf{x}\mid\boldsymbol{\alpha}) = \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1} \, d \mathbf{p}  $$
$$ = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} 
\prod_{j=1}^d p_j^{x_j + \alpha_j-1} \, d \mathbf{p}  $$
$$ = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}$$
$$ = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)} $$





## Q2

Given independent data points $\mathbf{x}_1, \ldots, \mathbf{x}_n$, show that the log-likelihood is
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
Is the log-likelihood a concave function?

We have already known that $ f(x) = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}$
$$ \begin{aligned}
L(\alpha) &= \ln \prod_{i=1}^n  f(\mathbf{x}_i \mid \alpha)\\
&=\sum_{i=1}^n\ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d \ln \Gamma(\alpha_j + x_{ij})- \sum_{i=1}^n \sum_{j=1}^d \ln \Gamma(\alpha_j)+\sum_{i=1}^n \ln \Gamma(|\alpha|)- \sum_{i=1}^n \ln \Gamma(|\alpha|+|\mathbf{x}_i|)\\
&= \sum_{i=1}^n\ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
\end{aligned} $$

To see if the log-likelihood is a concave function, the Hessian of L need to be calculated. (The function will be impliment in following question)

## Q3

Write Julia function to compute the log-density of the Dirichlet-multinomial distribution.

In [4]:
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    n = size(x)[1]
    d = size(α)

    l = 0.0
    
    # These parts are computed from each element of x
    # ∑[-ln(x_ij) + ln(Γ(α_j + x_{ij})) - ln(Γ(α_j))]
    for i = 1:n
         l += - lfact(x[i]) - lgamma(α[i]) + lgamma(x[i] + α[i])
    end

    # These parts are unrelated to each element of x
    # ln(|x|) - ln(Γ(|α| + |x|)) + ln(Γ(|α|))
    norm_x = sum(x)
    norm_α = sum(α)    
    l = lfact(norm_x) - lgamma(norm_α + norm_x) + lgamma(norm_α)

    return l
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

## Q4

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Each row contains the 64 counts of a digit and the last element (65th element) indicates what digit it is. For grading purpose, evaluate the total log-likelihood of this data at parameter values $\alpha=(1,\ldots,1)$ using your function in Q3.

In [5]:
# read in optdigits.tra
download("http://hua-zhou.github.io/teaching/" * 
    "biostatm280-2018spring/hw/hw4/optdigits.tra",
    "optdigits.tra")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  550k  100  550k    0     0  1287k      0 --:--:-- --:--:-- --:--:-- 1289k


"optdigits.tra"

In [6]:
opt = readcsv("optdigits.tra", Int)
x = opt[:, 1:64]' 
α = ones(64)
loglkh = dirmult_logpdf(x, α)

3823-element Array{Float64,1}:
 -165.188
 -176.23 
 -167.774
 -165.564
 -157.79 
 -176.071
 -158.423
 -159.258
 -174.302
 -178.407
 -171.294
 -169.383
 -175.753
    ⋮    
 -160.49 
 -158.633
 -156.935
 -171.975
 -177.638
 -169.735
 -158.423
 -159.465
 -159.877
 -169.735
 -173.149
 -155.411

In [7]:
# total log-likelihood 
tot_loglkh = sum(loglkh)

-638817.993292528

## Q5

Derive the score function $\nabla L(\alpha)$, observed information matrix $-d^2L(\alpha)$, and Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ for the Dirichlet-multinomial distribution.

Comment on why Fisher scoring method is inefficient for computing MLE in this example.

The score: 
$$\nabla L(\theta) = \begin{pmatrix}
  \frac{\partial L(\theta)}{\partial \theta_1} \\
  \vdots \\
  \frac{\partial L(\theta)}{\partial \theta_p}
  \end{pmatrix} $$

$$ \frac{\partial}{\partial \alpha_j} L(\alpha) = \frac{\partial}{\partial \alpha_j} (\sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)] )$$
$$ = \sum_{i=1}^n [\psi(x_{ij}+\alpha_j) - \psi(\alpha_j)] - \sum_{i=1}^n [\psi(|\mathbf{x}_i|+|\alpha|) - \psi(|\alpha|)]$$
where $\psi(x) = [\ln \Gamma(x)]' = \Gamma'(x)/\Gamma(x)$

The observed information matrix:
$$ \nabla^2 L(\theta) = d^2 L(\theta) = \begin{pmatrix}
  \frac{\partial^2 L(\theta)}{\partial \theta_1 \partial \theta_1} & \dots & \frac{\partial^2 L(\theta)}{\partial \theta_1 \partial \theta_p} \\
  \vdots & \ddots & \vdots \\
  \frac{\partial^2 L(\theta)}{\partial \theta_p \partial \theta_1} & \dots & \frac{\partial^2 L(\theta)}{\partial \theta_p \partial \theta_p}
  \end{pmatrix}$$
  
$$ [- d^2 L(\alpha)] = \sum_{i=1}^n [\psi'(\alpha_j) -\psi'(x_{ij}+\alpha_j)] - \sum_{i=1}^n [ \psi'(|\alpha|) + \psi'(|\mathbf{x}_i|+|\alpha|)] $$

To express the $[- d^2 L(\alpha)]$ simpler, observed information matrix can be shown as:
$$[- d^2 L(\alpha)] = \mathbf{D} - c \mathbf{1} \mathbf{1}^T,$$
where $ d_j = \sum_{i=1}^n \sum_{k=0}^{x_{ij}-1} \frac{1}{(\alpha_j + k)^2} $
and $ c = \sum_{i=1}^n \sum_{k=0}^{|\mathbf{x}_i|-1} \frac{1}{(|\alpha|+k)^2}$

The expected information matrix: 
$$\mathbf{E}[-d^2L(\alpha)] = \mathbf{E}[\mathbf{D} - c \mathbf{1} \mathbf{1}^T] = \mathbf{E}(\mathbf{D}) - c \mathbf{1}_d \mathbf{1}_d^T $$
$$ \mathbf{E}(d_j) = \mathbf{E}[\sum_{i=1}^n \sum_{k=0}^{x_{ij}-1} \frac{1}{(\alpha_j + k)^2}] = \sum_{i=1}^n \sum_{k=0}^{|\mathbf{x}_i|-1} \frac{\mathbf{P}(X_{ij}>k)}{(\alpha_j+k)^2},$$

Newton's method can be computed by matrix operations, but Fisher scoring method needs to compute Fisher information matrix which has $n\times d$ probabilities, which is inefficient in Julia.

## Q6

What structure does the observed information matrix possess that can facilitate the evaluation of the Newton direction? Is the observed information matrix always positive definite? What remedy can we take if it fails to be positive definite? (Hint: HW1 Q6.)

By the result of HW1 Q6 

Sherman-Morrison formula: $(\mathbf{A} + \mathbf{u} \mathbf{u}^T)^{-1} = \mathbf{A}^{-1} - \frac{1}{1 + \mathbf{u}^T \mathbf{A}^{-1} \mathbf{u}} \mathbf{A}^{-1} \mathbf{u} \mathbf{u}^T \mathbf{A}^{-1}$; 
Determinant formula: $\text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) = \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})$

$$[-d^2L(\boldsymbol{\alpha})]^{-1} = \boldsymbol{D}^{-1} + \frac{1}{c^{-1} - \sum_j d_j^{-1}} \boldsymbol{D}^{-1} \boldsymbol{1} \boldsymbol{1}^T \boldsymbol{D}^{-1} $$
$$ \text{det}[-d^2L(\boldsymbol{\alpha})] = \text{det}[\mathbf{D} - c \mathbf{1} \mathbf{1}^T],$$
$$ = \text{det}(\mathbf{D}) \text{det}(1 - c*\boldsymbol{D}^{-1}) $$
$$ = \left( \prod_j d_j \right) \left(1 - c \sum_j d_j^{-1} \right)$$

Therefore, the observed information matrix is not always positive definite, but only when $\text{det}[-d^2L(\boldsymbol{\alpha})] = \left( \prod_j d_j \right) \left(1 - c \sum_j d_j^{-1} \right) > 0$.

When the observed information matrix fails to be positive definite, $c$ can be adjusted to satisfy these two conditions:
1. each element of the diagonal matrix $\mathbf{D}$ is greater than 0
2. $c < \frac{1}{\sum_j d_j^{-1}} .$


## Q7

Discuss how to choose a good starting point. Implement this as the default starting value in your function below. (Hint: Method of moment estimator may furnish a good starting point.)

According Prof.Zhou's announcement:

$ P=(P_1, \ldots, P_d) $ is from Dirichlet distribution with parameter, $\alpha = (\alpha_1, \ldots, \alpha_d)$, then $ E(P_j) = \alpha_j / (\sum_{j’} \alpha_{j’}) $ and $ E(P_j^2) = \alpha_j (\alpha_j + 1) / (\sum_{j’} \alpha_{j’}) / ((\sum_{j’} \alpha_{j’}) + 1) $

Then we can get a equation for $\alpha$: 
$$ \frac{\mathbf{E}(P_j^2)}{\mathbf{E}(P_j)} = \frac{\boldsymbol{\alpha_j} + 1}{|\boldsymbol{\alpha}|+1} $$
then the sum $\alpha$ can be derive, 
$$ w := \sum_j \frac{\mathbf{E}(P_j^2)}{\mathbf{E}(P_j)} = \frac{|\boldsymbol{\alpha}|+d}{|\boldsymbol{\alpha}|+1} $$

$$ |\boldsymbol{\alpha}| = \frac{d - w}{w - 1} $$

Since $E(P_j)$ and $E(P_j^2)$ can be estimated by $\sum_i (\frac{x_{ij}}{|\boldsymbol{x_i}|})$ and $\sum_i (\frac{x_{ij}}{|\boldsymbol{x_i}|})^2$, respectively, $w$ can be estimated by:

$$ \hat{w} = \sum_j \frac{\sum_i (\frac{x_{ij}}{|\boldsymbol{x_i}|})^2}{\sum_i (\frac{x_{ij}}{|\boldsymbol{x_i}|})}$$

Then $\alpha_j$ can be initialized as

$$ \alpha_j = |\boldsymbol{\alpha}|\frac{\sum_i x_{ij}}{\sum_i |\boldsymbol{x_i}|} = \frac{d - \hat{w}}{\hat{w} - 1} \frac{\sum_i x_{ij}}{\sum_i |\boldsymbol{x_i}|}$$


## Q8

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using the Newton's method. 

In [41]:
# Before writing dirmult_newton implementation evaluate gradient (score)
function dirmult_gradient(x::Matrix{Float64}, α::Vector{Float64})
    n = size(x, 1)
    d = size(x, 2)
    grad = zeros(d)
    sum1 = sum(α)
    sum2 = sum(X, 2)
    cons = - (digamma(α) .- digamma(sum1))
    for i in 1:n
        for j in 1:d
            grad[i] += digamma(α[i] + X[i, j]) - digamma(sum1 + sum2[i]) + cons[i]
        end
    end
    return grad
end

dirmult_gradient (generic function with 1 method)

In [40]:
# observed information matrix
#Return vector and constant `con`. The observed information matrix equals `Diagonal(vector) - con`.
function dirmult_obs(X::Matrix, α::Vector)
    sum1 = sum(α)
    sum2 = sum(X, 2)
    n = size(x, 1)
    d = size(x, 2)
    vector = zeros(d)
    con = 0
    for i in 1:n
        con += trigamma(sum1 + sum2[i]) - trigamma(sum1)   
        vector[i] += trigamma(α[i]) - trigamma(x[i] + α[i])
    end
    return Diagonal(vector), con
end

dirmult_obs (generic function with 1 method)

In [2]:
"""
    dirmult_newton(X)

Find the MLE of Dirichlet-multinomial distribution using Newton's method.

# Argument
* `X`: an `n`-by-`d` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: a `d` vector of starting point (optional). 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
* `maximum`: the log-likelihood at MLE.   
* `estimate`: the MLE. 
* `gradient`: the gradient at MLE. 
* `hessian`: the Hessian at MLE. 
* `se`: a `d` vector of standard errors. 
* `iterations`: the number of iterations performed.
"""
function dirmult_newton(X::Matrix; α0::Vector = nothing, 
            maxiters::Int = 100, tolfun::Float64 = 1e-6)
    
    # set default starting point from Q7
    
    # Newton loop
    for iter in 1:maxiters
        
        # evaluate gradient (score)
        gradient = dirmult_gradient(X, α)
        
        # obs information matrix
        A, u = hess(Xnew, α0)
        
        # compute Newton's direction
        ndir = 
        # line search loop
        for lsiter in 1:10
            # step halving
        end
        
        # check convergence criterion
        if abs(logl - loglold) < tolfun * (abs(loglold) + 1)
            break;
        end
    end
    
    # compute logl, gradient, Hessian from final iterate
    
    # output
    
end

dirmult_newton

## Q9

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Find the MLE for the subset of digit 0, digit 1, ..., and digit 9 separately using your function. 

## Q10

As $\alpha / |\alpha| \to \mathbf{p}$, the Dirichlet-multinomial distribution converges to a multinomial with parameter $\mathbf{p}$. Therefore multinomial can be considered as a special Dirichlet-multinomial with $|\alpha|=\infty$. Perform a likelihood ratio test (LRT) whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.

## Q11

Now we can construct a simple Bayesian rule for handwritten digits recognition:
$$
	\mathbf{x}	\mapsto \arg \max_k \widehat \pi_k f(x|\widehat \alpha_k).
$$
Here we can use the proportion of digit $k$ in the training set as the prior probability $\widehat \pi_k$. Report the performance of your classifier on the test set of 1797 digits in `optdigits.tes`.